In [ ]:
import os
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F


import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams["animation.embed_limit"] = 128

import skimage
import skimage.io as sio
import skimage.transform

import yuca
from yuca.ca.neural import NCA
from yuca.ca.continuous import CCA
from yuca.ca.reaction_diffusion import RxnDfn
from yuca.cppn import CPPN

from yuca.zoo.librarian import Librarian

import IPython

from importlib import reload
reload(yuca)
reload(yuca.ca)

In [ ]:
def plot_grid(grid, my_cmap=plt.get_cmap("magma"), title="CA animation", vmin=0.0, vmax=1):

    global subplot_0
    
    fig, ax = plt.subplots(1,1, figsize=(4.5,4.5), facecolor="white")

    # TODO invert cmap
    
    grid_display = my_cmap(grid[0,0])[:,:,0:3]
    
    subplot_0 = ax.imshow(grid_display, interpolation="nearest")
    
    fig.suptitle(title, fontsize=8)

    ax.set_yticklabels('')
    ax.set_xticklabels('')
    
    plt.tight_layout()

    return fig, ax

def update_fig(i):

    global subplot_0    
    global grid
    global speedup
    
    my_cmap=plt.get_cmap("magma")
    
    for ii in range(speedup):
        grid = uskate(grid)
    
    grid_display = my_cmap(grid[0,0])[:,:,0:3]
    
    subplot_0.set_array(grid_display)
        
    plt.tight_layout()

In [ ]:
uskate = RxnDfn()

In [ ]:
lib = Librarian()
lib.index

In [ ]:
p, m = lib.load("uskate_berrycup001")

plt.figure()
plt.imshow(p[0,0])
plt.show()

In [ ]:
num_frames = 500
speedup=100

grid = uskate.initialize_grid()
grid[0,:,:p.shape[-2],:p.shape[-1]] = torch.tensor(p)

fig, ax = plot_grid(grid)

IPython.display.HTML(\
        matplotlib.animation.FuncAnimation(fig, update_fig, frames=num_frames, interval=10).to_jshtml())

In [ ]:
number_samples = 3
number_steps = 262144


my_dx_0 = uskate.get_dx()
min_dx = uskate.get_dx() / 2
max_dx = (uskate.get_dx() - min_dx) + my_dx_0

my_dxs = np.arange(min_dx, max_dx, \
                  (max_dx-min_dx) / number_samples)

my_dts = np.arange(1/number_samples,\
                   1.+ 1/number_samples, \
                   1/number_samples)

pattern_names = ["uskate_berrycup001",\
        "uskate_daedalus001",\
        "uskate_glider001"]

t0 = time.time()

for pattern_name in pattern_names:
    p, m = lib.load(pattern_name)
    for my_dt in my_dts:

        for my_dx in my_dxs:
            uskate = RxnDfn()
            uskate.set_dt(my_dt)
            uskate.set_dx(my_dx)

            grid = uskate.initialize_grid()
            grid[0,:,:p.shape[-2],:p.shape[-1]] = torch.tensor(p)

            t1 = time.time()
            for step in range(number_steps):

                grid = uskate(grid)

            t2 = time.time()

            my_dx *= 3

            print(f"run time {t2-t1:.3f}, total time elapsed: {t2-t0:.3f}")

            fig, ax = plot_grid(grid)

            fig.suptitle(f"dt = {uskate.get_dt():.3f}, dx ={uskate.get_dx():.3e} \n step {step}")
            plt.tight_layout()
            fig_name = f"{pattern_name}_dx{uskate.get_dt():.3f}_{uskate.get_dx():.3e}"
            fig_name = fig_name.replace(".","x") + ".png"
            fig_name = os.path.join("..","results", "uskate_results_assets", fig_name)
            print(f"saving to {fig_name}")
            plt.savefig(fig_name)

            plt.show()



In [ ]:
# make figure from saved end-states

uskate= RxnDfn()

number_samples = 20

my_dx_0 = uskate.get_dx()
min_dx = uskate.get_dx() / 50
max_dx = (uskate.get_dx() - min_dx) + my_dx_0

dxs = np.arange(min_dx, max_dx, \
                  (max_dx - min_dx) / number_samples)


ddt=0.04
dts = np.arange(ddt,1.0+ddt,ddt)
dts.shape, dxs.shape

dtdt, dxdx = np.meshgrid(dts,dxs)

berrycup_support = np.array([2.733e-3, 3.03e-3, 3.327e-3, 3.624e-3])
daedalus_support = np.array([2.733e-3, 3.03e-3, 3.327e-3, 3.624e-3])
uskate_support = np.array([2.733e-3, 3.03e-3, 3.327e-3, 3.624e-3, 3.921e-3])


pattern_names = ["uskate_daedalus001",\
        "uskate_glider001",\
        "uskate_berrycup001"]

pattern_titles = ["U-Skate Daedalus",\
                  "U-Skate U-Skate glider",\
                 "U-Skate BerryCup"]

t0 = time.time()

image_map = [None, None, None]

for hh, pattern_name in enumerate(pattern_names):
    
    image_map = [None, None, None]
    for ii, dt in enumerate(dts):
        for jj, dx in enumerate(dxs):
            
            fig_name = f"{pattern_name}_step262144_dx{dt:.3f}_{dx:.3e}"

            fig_name = fig_name.replace(".","x") + ".png"
            fig_name = os.path.join("..","results", "uskate_results_assets", fig_name)
            if os.path.exists(fig_name):
                img = sio.imread(fig_name)[68:-20,44:-44,:]
            else:
                print("f{fig_name} doesn't exist")
                img = np.zeros((362,362,4))
                
            img = skimage.transform.resize(img, [elem // 4 for elem in img.shape[:2]])
                
            if jj == 0:
                if ii > 0:
                    if image_map[hh] is None:
                        image_map[hh] = temp_img_map
                    else:
                        image_map[hh] = np.append(image_map[hh], temp_img_map, axis=1)
                    
                temp_img_map = img
            else:
                temp_img_map = np.append(temp_img_map, img, axis=0)
             
    fig, ax =  plt.subplots(1,1,figsize=(10,10))

    ax.imshow(image_map[hh])

    xtick_start = image_map[hh].shape[1] / 25 / 2
    ytick_start = image_map[hh].shape[0] / 20 / 2

    ax.set_xticks(np.arange(xtick_start,image_map[hh].shape[1], image_map[hh].shape[1] / 25))
    ax.set_yticks(np.arange(ytick_start,image_map[hh].shape[0], image_map[hh].shape[0] / 20))
    dt_labels = [f"{dt:.2f}" for dt in dts]
    dx_labels = [f"{dx:.3e}" for dx in dxs]
    ax.set_xticklabels(dt_labels, rotation=45)
    ax.set_yticklabels(dx_labels)

    ax.set_xlabel("$\Delta t$", fontsize=28)
    ax.set_ylabel("$\Delta x$", fontsize=28)
    
    ax.set_title(f"{pattern_titles[hh]}\n stability w.r.t $\Delta x$ and $\Delta t$", fontsize=36)


    save_to = os.path.join("..","assets", f"{pattern_name}_dxdt_summary.png")
    plt.savefig(save_to)

    plt.show()